## Accelerate Inference: Neural Network Pruning

In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.4.1


In [4]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [64]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [65]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 25, 25, 32)        896       
_________________________________________________________________
activation_24 (Activation)   (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 23, 23, 32)        9248      
_________________________________________________________________
activation_25 (Activation)   (None, 23, 23, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 11, 11, 64)       

In [66]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50
703/703 [==============================] - 4s 5ms/step - loss: 1.5806 - accuracy: 0.2593 - val_loss: 1.3823 - val_accuracy: 0.4028
Epoch 2/50
703/703 [==============================] - 3s 5ms/step - loss: 1.3890 - accuracy: 0.4025 - val_loss: 1.2784 - val_accuracy: 0.4483
Epoch 3/50
703/703 [==============================] - 3s 5ms/step - loss: 1.2961 - accuracy: 0.4537 - val_loss: 1.2148 - val_accuracy: 0.4947
Epoch 4/50
703/703 [==============================] - 3s 4ms/step - loss: 1.2390 - accuracy: 0.4833 - val_loss: 1.1695 - val_accuracy: 0.4998
Epoch 5/50
703/703 [==============================] - 3s 5ms/step - loss: 1.1925 - accuracy: 0.5030 - val_loss: 1.1507 - val_accuracy: 0.5220
Epoch 6/50
703/703 [==============================] - 3s 5ms/step - loss: 1.1692 - accuracy: 0.5201 - val_loss: 1.1025 - val_accuracy: 0.5509
Epoch 7/50
703/703 [==============================] - 3s 5ms/step - loss: 1.1293 - accuracy: 0.5419 - val_loss: 1.0656 - val_accuracy: 0.5659
Epoch 

In [67]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 4ms/step - loss: 0.7569 - accuracy: 0.7212


In [127]:
model.load_weights("my_model_weights.h5")

In [128]:
# perform pruning here

# get the weights 
weights = model.get_weights()
# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf
def prune(weight):
  weight = tf.cast(tf.math.abs(weight) > 1e-2, weight.dtype) * weight
  return weight
for i in range(len(weights)):
  weights[i] = prune(weights[i])
model.set_weights(weights)

In [129]:
# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 4ms/step - loss: 0.7391 - accuracy: 0.7232


In [111]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("my_model_weights.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
g